In [ ]:
from sklearn.preprocessing import StandardScaler
# get data module
from newsapi import NewsApiClient
import yfinance as yf
import csv
from datetime import datetime, timedelta

In [20]:
COMPANY_NAMES = ['Tesla']
tickers = {'Tesla':'TSLA'}
news_headlines = {} # dictionary in the form of news_headlines = {company name: {date: [title 1, title 2, etc.], date: [titles], ...}
prices = {}

## Get data

In [ ]:
def fetch_news(api_key, company_name, days=30):
    newsapi = NewsApiClient(api_key=api_key)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)

    news_data = {}

    current_date = start_date
    while current_date <= end_date:
        current_date_str = current_date.strftime('%Y-%m-%d')
        next_date = current_date + timedelta(days=1)
        all_articles = newsapi.get_everything(
            q=company_name,
            from_param=current_date_str,
            to=next_date.strftime('%Y-%m-%d'),
            language='en',
            sort_by='publishedAt',
            page_size=100
        )

        for article in all_articles['articles']:
            title = article['title']
            # date = article['publishedAt'].split('T')[0]
            # link = article['url']
            if current_date_str in news_data.keys():
                news_data[current_date_str].append(title)
            else:
                news_data[current_date_str] = [title]
            # print(f"Fetched article: {title}, Date: {date}")
            # {company name: {date: [title 1, title 2, etc.], date: [titles], ...}

        current_date = next_date

    # Print the oldest article
    # if news_data:
    #     oldest_article = min(news_data, key=lambda x: x['date'])
    #     print(f"Oldest article: {oldest_article['title']}, Date: {oldest_article['date']}")

    # with open(f'{company_name}_news.csv', mode='w', newline='', encoding='utf-8') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(['Title', 'Date', 'Link'])
    #     for news in news_data:
    #         writer.writerow([news['title'], news['date'], news['link']])
    
    # print("News titles and dates have been saved to tsla_news.csv")
    
    news_headlines[company] = news_data
    print("Titles have been saved to the dictionary")

# Replace 'YOUR_API_KEY' with your actual NewsAPI key
for company in COMPANY_NAMES:
    fetch_news(api_key='7e9676fa277d423f8deff9a4894bd717', company_name=company, days=30)
    
    # Define the ticker symbol and fetch data for the last 30 days
    ticker = tickers[company]
    data = yf.download(ticker, period="1mo", interval="1d")

    # Extract the adjusted closing price and convert to a pandas DataFrame
    prices_df = data[['Adj Close']].reset_index()
    prices_df.rename(columns={'Adj Close': 'Adjusted Close'}, inplace=True)
    
    # Flatten and rename
    prices_df.columns = prices_df.columns.get_level_values(-1)
    prices_df.columns = ['Date', 'Adjusted Close']
    prices[company] = prices_df

print(prices)
print(news_headlines)

Titles have been saved to the dictionary


[*********************100%***********************]  1 of 1 completed

{'Tesla':          Date  Adjusted Close
0  2024-10-23      213.649994
1  2024-10-24      260.480011
2  2024-10-25      269.190002
3  2024-10-28      262.510010
4  2024-10-29      259.519989
5  2024-10-30      257.549988
6  2024-10-31      249.850006
7  2024-11-01      248.979996
8  2024-11-04      242.839996
9  2024-11-05      251.440002
10 2024-11-06      288.529999
11 2024-11-07      296.910004
12 2024-11-08      321.220001
13 2024-11-11      350.000000
14 2024-11-12      328.489990
15 2024-11-13      330.239990
16 2024-11-14      311.179993
17 2024-11-15      320.720001
18 2024-11-18      338.739990
19 2024-11-19      346.000000
20 2024-11-20      342.029999
21 2024-11-21      339.640015
22 2024-11-22      352.559998}
{'Tesla': {'2024-10-25': ["Biden says Elon Musk was an 'illegal worker' when he began U.S. career", '‘Cowardice’: Jeff Bezos blocks Washington Post from endorsing Kamala Harris', 'Washington Post reports Elon Musk briefly worked illegally in US in 1990s', 'Washington P

In [15]:
print(news_headlines['Tesla']['2024-11-19'])

['Chinese cyberspies, Musk’s Beijing ties, labelled ‘real risk’ to US security by senator', 'Musk, Ramaswamy will lean on Supreme Court rulings to cut US agencies', "'Do You Sell Cars?': Tesla CEO Elon Musk Trolls Jaguar Rebrand on X", "7 key details from Elon Musk and Vivek Ramaswamy's DOGE plan", 'MSTU’s Bitcoin Leverage Spikes 900% in Two Months', 'Musk, Ramaswamy Push for Federal Workers to Return to Office Full Time', "Musk Said Bezos' Ex-Wife, MacKenzie Scott, Was a Reason 'Western Civilization Died'", 'Nvidia Crushes Earnings Expectations On AI Chip Demand', 'Nvidia crushes earnings expectations on AI chip demand', 'Nvidia crushes earnings expectations on AI chip demand', 'Nvidia crushes earnings expectations on AI chip demand', 'Nvidia releases earnings: Takeaways from Mag 7 Q3 results', 'Meet the updated 2026 Genesis Electrified GV70, now with more range and an NACS port', "Trump allies' company stock prices soar after election", 'Trump administration paves the way for autonom

## Sentiment Analysis

In [22]:
import numpy as np
import keras
from keras import utils
from keras import models
from keras import layers
# from keras import preprocessing
import tensorflow as tf

# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
import re

import pandas as pd

df = pd.read_parquet("hf://datasets/TimKoornstra/financial-tweets-sentiment/data/train-00000-of-00001.parquet")

print(df.columns)
df.drop(columns='url')

print(f"tweets in dataset: {len(np.unique(df['tweet']))}")
#print(df.head)

pattern = r'[^\w\s]'

# Use regex substitution to remove special characters from the 'text' column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(pattern, '', x))

# Print the updated DataFrame
#print([(np.unique(df["tweet"]))])

# Use ASCII filtering to remove non-ASCII characters from the 'text' column
df['tweet'] = df['tweet'].apply(lambda x: ''.join(char for char in x if ord(char) < 128))
#print('\n\n\n')
#print(([(np.unique(df["tweet"]))]))
print(len(df['tweet']))


def remove_links(tweet):
    words = tweet.split()  # Split tweet into words
    filtered_words = [word for word in words if not word.startswith('http')]  # Remove words that start with 'http'
    return ' '.join(filtered_words)  # Join words back together into a tweet

# Apply the function to each entry in the 'tweets' column
df['tweet'] = df['tweet'].apply(remove_links)
df=df.drop(columns=['url'])
#print(df.head)

def remove_numerical_words(tweet):
    # Use regex to remove words containing any digits
    return ' '.join(word for word in tweet.split() if not re.search(r'\d', word))

# Apply the function to the 'tweet' column
df['tweet'] = df['tweet'].apply(remove_numerical_words)

# Check the updated DataFrame
print(df.head())
df.to_csv('out.csv', index=False) 

all_tweets = ' '.join(df['tweet'])

# Step 2: Split the combined string into individual words
all_words = all_tweets.split()

# Step 3: Use a set to get unique words
unique_words = set(all_words)

# Convert the set back to a sorted list (optional)
unique_words_list = sorted(list(unique_words))
print(f'unique words in dataset: {len(unique_words_list)}')
#print(unique_words_list)
#"Tokenizing" data

#df['sentiment'] = to_categorical(df['sentiment'].values, num_classes=3)

# Tokenizing and padding sequences (same as before)
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000)
tokenizer.fit_on_texts(df['tweet'])

sequences = tokenizer.texts_to_sequences(df['tweet'])
print(sequences[:50])
print(f'sequence length {len(sequences)}')
padded_sequences = utils.pad_sequences(sequences, padding='post')
max_sequence_length = padded_sequences.shape[1]

# Split the data into training and test sets
train_x = padded_sequences[:15000]
train_y = df['sentiment'][:15000]
test_x = padded_sequences[15000:]
test_y = df['sentiment'][15000:]

#turns this into one hot encoding format
train_y = utils.to_categorical(train_y, num_classes=3)
test_y = utils.to_categorical(test_y, num_classes=3)

print(train_y[1])


vocab_size=50000
# Training the model
model = models.Sequential()

# Embedding layer (input length is max_sequence_length after padding)
model.add(layers.Embedding(input_dim=vocab_size, output_dim=50, input_length=max_sequence_length))

# Flatten the output of the Embedding layer
model.add(layers.GlobalMaxPooling1D())

# Hidden Layers
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(50, activation="relu"))

# Output Layer (3 units for 3 classes, with softmax activation)
model.add(layers.Dense(3, activation="softmax"))

model.summary()

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
results = model.fit(
    train_x, train_y,
    epochs=5,
    batch_size=32,
    validation_data=(test_x, test_y)
)

train_accuracy = results.history['accuracy']
val_accuracy = results.history['val_accuracy']
train_loss = results.history['loss']
val_loss = results.history['val_loss']



#Testing on new data
new_data = pd.DataFrame({
    'tweet': [
        "JPMorgan price will fall. It's been a bad week for the company", 
        "Stock price soars on great day production increases at the company.", 
        "I am going to be looking at the stock price later today.",
        "There has been a change in management at chipotle.",
        "Profits soar for small startup with a big jump in sales."
    ]
})

print(f"Max sequence length from training data: {max_sequence_length}")

# Step 1: Preprocess the new data
# Clean the new data
pattern = r'[^\w\s]'  # Keep only words and spaces
new_data['tweet'] = new_data['tweet'].apply(lambda x: re.sub(pattern, '', x))

# Step 2: Tokenize the new data
# Use the same tokenizer that was fitted on the training data
sequences_new = tokenizer.texts_to_sequences(new_data['tweet'])

# Step 3: Pad the sequences (using the defined max_sequence_length)
padded_sequences_new = utils.pad_sequences(sequences_new, maxlen=max_sequence_length, padding='post')

# Step 4: Make predictions

predictions = model.predict(padded_sequences_new)
print(predictions)
# Step 5: Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Output the results
for tweet, sentiment in zip(new_data['tweet'], predicted_classes):
    sentiment_labels = ['Neutral', 'Positive', 'Negative']
    print(f"Tweet: '{tweet}' => Predicted Sentiment: {sentiment_labels[sentiment]}")

Index(['tweet', 'sentiment', 'url'], dtype='object')
tweets in dataset: 38089
38091
                                               tweet  sentiment
0  BYND JPMorgan reels in expectations on Beyond ...          2
1  CCL RCL Nomura points to bookings weakness at ...          2
2  CX Cemex cut at Credit Suisse JP Morgan on wea...          2
3                  ESS BTIG Research cuts to Neutral          2
4       FNKO Funko slides after Piper Jaffray PT cut          2
unique words in dataset: 46650
[[2358, 1058, 7042, 5, 744, 9, 1297, 2742], [1090, 4300, 4576, 302, 2, 9656, 1298, 12, 3332, 6, 1825, 3850], [12178, 17168, 318, 12, 405, 1599, 1944, 728, 9, 667, 850, 610], [6328, 4577, 632, 510, 2, 915], [8063, 5763, 2622, 47, 2525, 4899, 621, 318], [7043, 12179, 1136, 12, 9657, 35, 593, 146, 906, 12, 1222, 208], [147, 147, 1600, 3, 444], [147, 1222, 208, 510, 2, 189], [8064, 4301, 510, 2, 45, 2526], [17169, 17170, 51, 12180, 510, 916, 1670, 47, 4900, 2359], [4302, 5292, 1441, 9, 17171], [17172

c:\Users\agaug\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 45s 87ms/step - accuracy: 0.5358 - loss: 0.9749 - val_accuracy: 0.4491 - val_loss: 1.1847
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 33s 70ms/step - accuracy: 0.8191 - loss: 0.4777 - val_accuracy: 0.4539 - val_loss: 1.4090
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 72ms/step - accuracy: 0.9246 - loss: 0.2320 - val_accuracy: 0.4739 - val_loss: 1.5379
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 70ms/step - accuracy: 0.9640 - loss: 0.1136 - val_accuracy: 0.4752 - val_loss: 2.0167
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 28s 60ms/step - accuracy: 0.9809 - loss: 0.0598 - val_accuracy: 0.4675 - val_loss: 2.5933
Max sequence length from training data: 688
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
[[8.7330118e-02 3.4551653e-03 9.0921474e-01]
 [3.4646359e-03 9.9355596e-01 2.9794031e-03]
 [2.3559919e-03 9.9650061e-01 1.1434335e-03]
 [9.9882942e-01 1.1037290e-03 6.6756409e-05]
 [3.9184004e-02 9.5993030e-01 8.8567240e-04]]
Tweet: 'JPMorgan price will fall Its been a ba

## Data Preprocessing

In [62]:
#Apply to news headlines
def standard_scaler():
    return

train_datasets = {}
test_datasets = {}
scalers = {}

sentiment_vectors = {}
for company in news_headlines:
    sentiment_vectors[company] = {'Date': [], 'polarity': []}
    for k, v in news_headlines[company].items():
        # print(datetime.strptime(k, r'%Y-%m-%d').weekday())
        if datetime.strptime(k, r'%Y-%m-%d').weekday() >= 5:
            continue
        headlines = pd.DataFrame({k: v})

        # Step 1: Preprocess the new data
        # Clean the new data
        pattern = r'[^\w\s]'  # Keep only words and spaces
        headlines[k] = headlines[k].apply(lambda x: re.sub(pattern, '', x))
        
        # Step 2: Tokenize the new data
        # Use the same tokenizer that was fitted on the training data
        sequences_new = tokenizer.texts_to_sequences(headlines[k])

        # Step 3: Pad the sequences (using the defined max_sequence_length)
        padded_sequences_new = utils.pad_sequences(sequences_new, maxlen=max_sequence_length, padding='post')

        # Step 4: Make predictions

        predictions = model.predict(padded_sequences_new)
        predictions_array = np.array(predictions)
        mean_pred = np.mean(predictions_array, axis=0)
        neutral = mean_pred[0]
        positive = mean_pred[1]
        negative = mean_pred[2]
        
        # find the polarity
        polarity = (positive - negative) / (negative + positive + neutral)
        sentiment_vectors[company]['Date'].append(k)
        # sentiment_vectors[company]['positive'].append(positive)
        # sentiment_vectors[company]['neutral'].append(neutral)
        # sentiment_vectors[company]['negative'].append(negative)
        sentiment_vectors[company]['polarity'].append(polarity)
        
        # Average the weekend vectors and assign that value to the friday date
        sentiment_df = pd.DataFrame(sentiment_vectors[company])

    # print(sentiment_df)
    
    # Call the correct prices dataframe
    prices_df = prices[company]
    
    # Change to Date column to datetime type
    sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'])
    prices_df['Date'] = pd.to_datetime(prices_df['Date'])
    
    # Merge the DataFrames on the "Date" column
    merged_df = pd.merge(sentiment_df, prices_df, how='left', on='Date')
    print(merged_df)
    
    # Z scale data
    std_scaler = StandardScaler()
    mean = merged_df.mean()
    std = merged_df.std()
    df_scaled = std_scaler.fit_transform(merged_df.iloc[:,1:].to_numpy())
    df_scaled = pd.DataFrame(df_scaled, columns=['polarity', 'Adjusted Close'])
    df_scaled["Date"] = merged_df['Date']
    

    # set target to the upcoming closing prices, and remove the last row
    time_steps = 1
    y = df_scaled['Adjusted Close']
    for i in range(time_steps):
        y = y.drop(index=df_scaled.index[i])
    y = y.rename('Next Day Adjusted Close')
    df_scaled = df_scaled.iloc[:-time_steps]
    print(y)
    print(df_scaled)
    
    # Split 8:2 training and validation
    split_idx = int(len(df_scaled) *0.8)
    train_data_X = df_scaled.iloc[:split_idx]
    test_data_X = df_scaled.iloc[split_idx:]
    train_data_y = y.iloc[:split_idx]
    test_data_y = y.iloc[split_idx:]

    test_datasets[company] = (test_data_X, test_data_y)
    train_datasets[company] = (train_data_X, train_data_y)
    scalers[company] = (mean, std)
    
    

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 382ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
         Date  polarity  Adjusted Close
0  2024-10-25  0.133364      269.190002
1  2024-10-28  0.020947      262.510010
2  2024-10-29  0.106371      259.519989
3  2024-10-30 -0.019629      257.549988
4

## LSTM model

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from keras.models import Sequential
# from keras.layers import Dense, LSTM
# from sklearn.preprocessing import MinMaxScaler
import os
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras import callbacks
# from keras.callbacks import ReduceLROnPlateau

# Additional features

# Function to create lag features and rolling statistics
def create_lag_features(data, lags):
    for lag in range(1, lags + 1):
        data[f'lag_{lag}'] = data['Adjusted Close'].pct_change(lag)  # Percentage change
    data['rolling_mean'] = data['Adjusted Close'].rolling(window=5).mean()
    data['rolling_std'] = data['Adjusted Close'].rolling(window=5).std()
    return data

# Function to train the LSTM model with ReduceLROnPlateau scheduler
def train_lstm(X_train, y_train, input_shape):
    model = models.Sequential()
    model.add(layers.LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(layers.LSTM(units=50, return_sequences=False))  # Add GRU layer
    model.add(layers.Dense(units=25))
    model.add(layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Reshape input data for LSTM
    X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

    # Reduce learning rate on plateau callback
    lr_scheduler = callbacks.ReduceLROnPlateau(
        monitor='loss',          # Metric to monitor (use 'val_loss' if you have validation data)
        factor=0.5,              # Factor by which to reduce the learning rate (new_lr = lr * factor)
        patience=5,              # Number of epochs with no improvement before reducing learning rate
        verbose=1,               # Print messages when reducing the learning rate
        min_lr=1e-6              # Minimum learning rate
    )

    # Train the model with the learning rate scheduler
    model.fit(
        X_train, y_train.values,
        epochs=50,
        batch_size=32,
        verbose=1,
        callbacks=[lr_scheduler]  # Add the ReduceLROnPlateau scheduler
    )
    return model

# Function to evaluate and visualize predictions for multiple starting points
def evaluate_multiple_start_points(model, X_test, y_test, n_days_to_predict):
    start_indices = [0]#range(0, 30, 5)  # Manually specify starting indices
    for start_idx in start_indices:
        end_idx = start_idx + n_days_to_predict

        # Ensure the model always gets the past 9 days of data
        context_idx = start_idx - 9
        X_context = X_test.iloc[context_idx:start_idx]  # Past 9 days
        X_sub = X_train.iloc[-9:] # recheck on start_25
        print("xs",X_sub)
        y_true = y_test.iloc[start_idx:end_idx]

        # Reshape for LSTM input
        X_sub = X_sub.values.reshape((X_sub.shape[0], X_sub.shape[1], 1))
        print("xs",X_sub.shape)

        predictions = model.predict(X_sub)
        print(predictions.shape)
        # predictions = target_scaler.inverse_transform(predictions.reshape(-1, 1)).ravel()
        

        # Plot predictions
        plt.figure(figsize=(10, 6))

        plt.plot(np.arange(len(predictions)), predictions, label="Predictions", color="orange")
        plt.xlabel("Date")
        plt.ylabel("Close Price")
        plt.title(f"Predictions from Index {start_idx} to {end_idx}")
        plt.legend()
        # os.makedirs("../TSLA_pred/lstm", exist_ok=True)
        # plt.savefig(f"../TSLA_pred/lstm/future.png")
        plt.show()
        plt.close()


training_period = 9  # Number of days for lag features
n_days_to_predict = 9  # Number of days to predict for each starting point
num_start_points = 9  # Number of unique starting points

X_train = train_datasets['Tesla'][0]
X_test = test_datasets['Tesla'][0]
y_train = train_datasets['Tesla'][1]
y_test = test_datasets['Tesla'][1]

# Split the dataset into training and testing sets
# X_train, y_train, X_test, y_test, test_index, feature_scaler, target_scaler = split_data(df, training_period=1, idx = -1)
X_train = pd.concat((X_train, X_test))
y_train = pd.concat((y_train, y_test))
# Ensure X_test is an empty DataFrame with the same columns as X_train
X_test = pd.DataFrame(columns=X_train.columns)

# Ensure y_test is an empty Series with the same name as the original y_test
y_test = pd.Series(name=y_test.name)

# Train the LSTM model
input_shape = (X_train.shape[1], 1)  # Number of features as input shape
model = train_lstm(X_train, y_train, input_shape)

# Evaluate predictions for multiple starting points
evaluate_multiple_start_points(model, X_test, y_test, n_days_to_predict)

c:\Users\agaug\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).